In [71]:
# Importing the required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Downloading contents of the web page
url = "https://drugcheckingbc.ca/results/"
data = requests.get(url).text

# Creating BeautifulSoup object
soup = BeautifulSoup(data, 'html.parser')

# Find tables
table = soup.find('table')

# Defining of the dataframe
df = pd.DataFrame(columns=['DATE', 'SITE', 'CITY/TOWN', 'EXPECTED SUBSTANCE', 'FENTANYL TEST STRIP', 'BENZO TEST STRIP', 'FTIR SPECTROMETER'])

# Collecting data
for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        DATE = columns[0].text.strip()
        SITE = columns[1].text.strip()
        CITY_TOWN = columns[2].text.strip()
        EXPECTED_SUBSTANCE = columns[3].text.strip()
        FENTANYL_TEST_STRIP = columns[4].text.strip()
        BENZO_TEST_STRIP = columns[5].text.strip()
        FTIR_SPECTROMETER = str(columns[6])[59:-5].split('<br/>')

        row = [{'DATE': DATE,  'SITE': SITE, 'CITY/TOWN': CITY_TOWN, 'EXPECTED SUBSTANCE': EXPECTED_SUBSTANCE, 'FENTANYL TEST STRIP': FENTANYL_TEST_STRIP, 'BENZO TEST STRIP': BENZO_TEST_STRIP, 'FTIR SPECTROMETER': FTIR_SPECTROMETER}]
        df_new = pd.DataFrame.from_records(row)
        df = pd.concat([df, df_new])
        
df = df.reset_index(drop=True)

# make one column per drug
results = pd.DataFrame(df['FTIR SPECTROMETER'].values.tolist()).add_prefix('FTIR SPECTROMETER ')
df = pd.concat([df,results], axis=1).drop(columns='FTIR SPECTROMETER')  

df.head()

,DATE,SITE,CITY/TOWN,EXPECTED SUBSTANCE,FENTANYL TEST STRIP,BENZO TEST STRIP,FTIR SPECTROMETER 0,FTIR SPECTROMETER 1,FTIR SPECTROMETER 2,FTIR SPECTROMETER 3
0,2022-11-30,Outreach Urban Health,Kelowna,Down (Unknown Opioid),Positive,Positive,Erythritol,Caffeine,Fentanyl,None
1,2022-11-30,Burdock House (Penticton),Penticton,Crack Cocaine,Negative,N/A,Cocaine Base,None,None,None
2,2022-11-30,Get Your Drugs Tested,Vancouver,Unknown,Negative,Negative,MDA,Uncertain Match,None,None
3,2022-11-30,Get Your Drugs Tested,Vancouver,GHB,Negative,N/A,GHB (Wet),Uncertain Match,None,None
4,2022-11-30,Insite,Vancouver,Methamphetamine,Negative,N/A,Methamphetamine,Dimethyl Sulfone,None,None


In [56]:
# loading more rows

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://drugcheckingbc.ca/results/")

button = driver.find_element(By.CSS_SELECTOR, ".sbcd-filters-title")
driver.execute_script("arguments[0].click();", button)

In [4]:
!pip install pyyaml

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------- 151.6/151.6 kB 62.4 kB/s eta 0:00:00


In [3]:
import pickle
import pandas as pd

result_lists = pickle.load(open(r"C:\Users\User\OneDrive\Desktop\Documents\06 Education\06 LHL\projects\drug-testing-project\data_files\df_lists-2023-01-03.pkl", 'rb'))

df = result_lists[0]

for i in range(len(result_lists[1:])):
    df = pd.concat([df, result_lists[i]])

In [4]:
df

,VISIT DATE,CITY,SITE,EXPECTED DRUG,CATEGORY,COLOUR,TEXTURE,FENTANYL STRIP,BENZO STRIP,FTIR SPECTROMETER
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-11-30,Vancouver,Get Your Drugs Tested,Methamphetamine,Stimulant,Colourless,Crystal,Neg,Neg,Methamphetamine
2,2022-11-30,Vancouver,Insite,Ketamine,Psychedelic,Colourless,Crystal,Neg,NaN,Ketamine
3,2022-11-30,Vancouver,Insite,Fentanyl,Opioid,Purple (light),Pebble,Pos,Neg,"Caffeine, Fentanyl, Erythritol"
4,2022-11-30,Vancouver,Get Your Drugs Tested,Down (Unknown Opioid),Opioid,Purple,Powder,Pos,Neg,"Caffeine, Fentanyl, Erythritol"
...,...,...,...,...,...,...,...,...,...,...
16,2019-12-02,Vancouver,SisterSpace,Down (Unknown Opioid),Opioid,Brown (dark),Powder,Pos,Neg,"Polyethylene glycol, Inositol, Fentanyl, Heroi..."
17,2019-12-02,Kamloops,The Branch,Cocaine,Stimulant,White,Powder,Neg,NaN,Cocaine hcl
18,2019-12-02,Kamloops,The Branch,Fentanyl,Opioid,Purple (light),Pebble,Pos,NaN,"Mannitol, Caffeine"
19,2019-12-02,Kamloops,The Branch,Down (Unknown Opioid),Opioid,Pink (light),Chunk,Pos,NaN,"Mannitol, Fentanyl, Caffeine"


In [5]:
df.shape

(52542, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52542 entries, 0 to 20
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   VISIT DATE         50040 non-null  object
 1   CITY               50040 non-null  object
 2   SITE               50040 non-null  object
 3   EXPECTED  DRUG     50040 non-null  object
 4   CATEGORY           50040 non-null  object
 5   COLOUR             50040 non-null  object
 6   TEXTURE            50040 non-null  object
 7   FENTANYL  STRIP    48405 non-null  object
 8   BENZO  STRIP       24243 non-null  object
 9   FTIR SPECTROMETER  49506 non-null  object
dtypes: object(10)
memory usage: 4.4+ MB


In [7]:
df.describe()

,VISIT DATE,CITY,SITE,EXPECTED DRUG,CATEGORY,COLOUR,TEXTURE,FENTANYL STRIP,BENZO STRIP,FTIR SPECTROMETER
count,50040,50040,50040,50040,50040,50040,50040,48405,24243,49506
unique,1062,33,96,368,7,32,15,3,3,7016
top,2021-07-31,Vancouver,Get Your Drugs Tested,Down (Unknown Opioid),Opioid,White,Powder,Neg,Neg,Mdma
freq,169,41489,35389,8558,18968,18916,12998,30777,16914,4142


In [9]:
df.tail()

,VISIT DATE,CITY,SITE,EXPECTED DRUG,CATEGORY,COLOUR,TEXTURE,FENTANYL STRIP,BENZO STRIP,FTIR SPECTROMETER
16,2019-12-02,Vancouver,SisterSpace,Down (Unknown Opioid),Opioid,Brown (dark),Powder,Pos,Neg,"Polyethylene glycol, Inositol, Fentanyl, Heroi..."
17,2019-12-02,Kamloops,The Branch,Cocaine,Stimulant,White,Powder,Neg,NaN,Cocaine hcl
18,2019-12-02,Kamloops,The Branch,Fentanyl,Opioid,Purple (light),Pebble,Pos,NaN,"Mannitol, Caffeine"
19,2019-12-02,Kamloops,The Branch,Down (Unknown Opioid),Opioid,Pink (light),Chunk,Pos,NaN,"Mannitol, Fentanyl, Caffeine"
20,2019-12-02,Kamloops,The Branch,Fentanyl,Opioid,Purple,Granules,Pos,NaN,"Caffeine, Fentanyl, Xylitol"
